In [94]:
#Loading Data
import os
from pathlib import Path
import shutil

# Define paths
original_dataset_dir = "../Project4/images/Images"
base_dir = "../Project4/split_data"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")
#Validation set for overfitting of complex models
val_dir = os.path.join(base_dir, "val")
# Ensure directories exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Define classes
classes = os.listdir(original_dataset_dir)
print(classes)

['n02091831-Saluki', 'n02093991-Irish_terrier', 'n02108422-bull_mastiff', 'n02086240-Shih-Tzu', 'n02113023-Pembroke', 'n02094114-Norfolk_terrier', 'n02086079-Pekinese', 'n02099849-Chesapeake_Bay_retriever', 'n02101388-Brittany_spaniel', 'n02105855-Shetland_sheepdog', 'n02100236-German_short-haired_pointer', 'n02104365-schipperke', 'n02110185-Siberian_husky', 'n02097209-standard_schnauzer', 'n02105251-briard', 'n02107574-Greater_Swiss_Mountain_dog', 'n02093859-Kerry_blue_terrier', 'n02108089-boxer', 'n02086646-Blenheim_spaniel', 'n02087046-toy_terrier', 'n02091467-Norwegian_elkhound', 'n02095570-Lakeland_terrier', 'n02115641-dingo', 'n02113978-Mexican_hairless', 'n02115913-dhole', 'n02108000-EntleBucher', 'n02109047-Great_Dane', 'n02110958-pug', 'n02090622-borzoi', 'n02111500-Great_Pyrenees', 'n02096437-Dandie_Dinmont', 'n02104029-kuvasz', 'n02091134-whippet', 'n02116738-African_hunting_dog', 'n02109525-Saint_Bernard', 'n02093754-Border_terrier', 'n02099712-Labrador_retriever', 'n020942

In [41]:
# Create directories for each class in train and test directories
for class_name in classes:
    class_train_dir = os.path.join(train_dir, class_name)
    class_test_dir = os.path.join(test_dir, class_name)
    class_val_dir = os.path.join(val_dir, class_name)
    os.makedirs(class_train_dir, exist_ok=True)
    os.makedirs(class_test_dir, exist_ok=True)
    os.makedirs(class_val_dir, exist_ok=True)

In [42]:
from random import shuffle

split_ratio_train = 0.7  # 70% train
split_ratio_val = 0.15  # 15% validation (from train data)
split_ratio_test = 0.15  # 15% test

for class_name in classes:
  class_images = os.listdir(os.path.join(original_dataset_dir, class_name))
  shuffle(class_images)  # Randomize image order

  num_images = len(class_images)

  # Calculate split points (adjust ratios as needed)
  num_train_images = int(split_ratio_train * num_images)
  num_val_images = int(split_ratio_val * num_images)
  num_test_images = num_images - num_train_images - num_val_images

  # Train set
  for image_name in class_images[:num_train_images]:
    src = os.path.join(original_dataset_dir, class_name, image_name)
    dst = os.path.join(train_dir, class_name, image_name)
    shutil.copyfile(src, dst)

  # Validation set (from train data)
  for image_name in class_images[num_train_images:num_train_images+num_val_images]:
    src = os.path.join(original_dataset_dir, class_name, image_name)
    dst = os.path.join(val_dir, class_name, image_name)
    shutil.copyfile(src, dst)

  # Test set
  for image_name in class_images[num_train_images+num_val_images:]:
    src = os.path.join(original_dataset_dir, class_name, image_name)
    dst = os.path.join(test_dir, class_name, image_name)
    shutil.copyfile(src, dst)

print("Data split and directories created successfully.")

Data split and directories created successfully.


In [43]:
#Check the splits worked
# Count the total number of images in train and test directories
#Using underscores to save memory by not storing labels,path, or directories and simply count number of files in each sub directory
total_train_count = sum(len(files) for _, _, files in os.walk(train_dir))
total_test_count = sum(len(files) for _, _, files in os.walk(test_dir))
total_val_count = sum(len(files) for _, _, files in os.walk(val_dir))
total_images = total_test_count+total_train_count+total_val_count
train_per = total_train_count/total_images *100
test_per = total_test_count/total_images*100
val_per = total_val_count/total_images *100

print(f"Total Images: {total_images}")
print(f"Total Train Images: {total_train_count}, Percent of Total: {train_per}")
print(f"Total Test Images: {total_test_count}, Percent of Total: {test_per}")
print(f"Total Validation Images: {total_val_count}, Percent of Total: {val_per}")
print("Split is Effective!")
classes_val = os.listdir(val_dir)
classes_test = os.listdir(test_dir)
classes_train = os.listdir(train_dir)

Total Images: 20580
Total Train Images: 14355, Percent of Total: 69.75218658892129
Total Test Images: 3200, Percent of Total: 15.54907677356657
Total Validation Images: 3025, Percent of Total: 14.698736637512146
Split is Effective!


In [51]:
#Train preprocessing
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
train_data_dir = '../Project4/split_data/train'

batch_size = 32
# target image size
img_height = 224
img_width = 224

# note that subset="training", "validation", "both", and dictates which dataset is returned
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
train_data_dir,
validation_split=0.2,
subset="both",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size
)
rescale = Rescaling(scale=1.0/255)
train_rescale_ds = train_ds.map(lambda image,label:(rescale(image),label))
val_rescale_ds = val_ds.map(lambda image,label:(rescale(image),label))

Found 14355 files belonging to 120 classes.
Using 11484 files for training.
Using 2871 files for validation.


In [52]:
#Test preprocessing
test_data_dir = '../Project4/split_data/test/'

batch_size = 2

# this is what was used in the paper --
img_height = 224
img_width = 224

# note that subset="training", "validation", "both", and dictates what is returned
test_ds = tf.keras.utils.image_dataset_from_directory(
test_data_dir,
seed=123,
image_size=(img_height, img_width),
)

# approach 1: manually rescale data --
rescale = Rescaling(scale=1.0/255)
test_rescale_ds = test_ds.map(lambda image,label:(rescale(image),label))

Found 3200 files belonging to 120 classes.


In [44]:
#Now on to training our models!
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Add, Input, Activation

In [87]:
#First Testing pre-trained ResNet50
from keras.applications import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

# Load pre-trained ResNet50 model
ResNet = ResNet50(weights='imagenet')
# Remove the existing top layer
x = ResNet.output

# Add a new classification layer
num_classes = 120  # Change this to your desired number of output classes
x = Dense(num_classes, activation='softmax')(x)

# Create a new model with your modified classification layer
ResNet = Model(inputs=ResNet.input, outputs=x)

ResNet.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

ResNet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_7[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [102]:
# Load an image for prediction
img_path = '../Project4/split_data/test/n02091032-Italian_greyhound/n02091032_1360.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Make predictions
predictions = model.predict(x)

# Decode and print the top-3 predicted classes
print('Predicted:', decode_predictions(predictions, top=3)[0])


1/1 [==============================] - 0s 192ms/step
Predicted: [('n02091032', 'Italian_greyhound', 0.81232905), ('n02091134', 'whippet', 0.18761577), ('n02091831', 'Saluki', 2.4743607e-05)]


In [89]:
#fit the Model (This model will take forever)
history = ResNet.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
  6/359 [..............................] - ETA: 23:36 - loss: 4.7877 - accuracy: 0.0052  

KeyboardInterrupt: 

In [47]:
#model AlexNet
AlexNet = models.Sequential()

# Layer 1: Convolutional layer with 96 filters of size 11x11, followed by max pooling
AlexNet.add(layers.Conv2D(96, kernel_size=(11,11),strides=4, activation='relu', padding='same', input_shape=(224, 224, 3)))
AlexNet.add(layers.MaxPooling2D(pool_size=(3, 3),strides=2))

AlexNet.add(layers.Conv2D(256, kernel_size=(5,5),padding='same', activation='relu'))
AlexNet.add(layers.MaxPooling2D(pool_size=(3, 3),strides=2))

AlexNet.add(layers.Conv2D(384, kernel_size=(3,3),padding='same', activation='relu'))
AlexNet.add(layers.Conv2D(384, kernel_size=(3,3),padding='same', activation='relu'))
AlexNet.add(layers.Conv2D(256, kernel_size=(3,3),padding='same', activation='relu'))

AlexNet.add(layers.MaxPooling2D(pool_size=(3, 3),strides=2))

# Flatten the feature maps to feed into fully connected layers
AlexNet.add(layers.Flatten())


# Layer 3: Fully connected layer with 120 neurons
AlexNet.add(layers.Dense(4096, activation='relu'))
AlexNet.add(layers.Dropout(.5))
# Layer 4: Fully connected layer with 84 neurons
AlexNet.add(layers.Dense(4096, activation='relu'))
AlexNet.add(layers.Dropout(.5))

# Output layer: Fully connected layer with num_classes neurons (e.g., 3 )
AlexNet.add(layers.Dense(120, activation='softmax'))

# Compile model
AlexNet.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
AlexNet.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_49 (Conv2D)          (None, 56, 56, 96)        34944     
                                                                 
 max_pooling2d_47 (MaxPooli  (None, 27, 27, 96)        0         
 ng2D)                                                           
                                                                 
 conv2d_50 (Conv2D)          (None, 27, 27, 256)       614656    
                                                                 
 max_pooling2d_48 (MaxPooli  (None, 13, 13, 256)       0         
 ng2D)                                                           
                                                                 
 conv2d_51 (Conv2D)          (None, 13, 13, 384)       885120    
                                                                 
 conv2d_52 (Conv2D)          (None, 13, 13, 384)     

In [9]:
#fit the model from image generator
history = AlexNet.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
411/411 [==============================] - 309s 749ms/step - loss: 4.7867 - accuracy: 0.0090 - val_loss: 4.7836 - val_accuracy: 0.0113
Epoch 2/20
411/411 [==============================] - 304s 740ms/step - loss: 4.7608 - accuracy: 0.0124 - val_loss: 4.6992 - val_accuracy: 0.0198
Epoch 3/20
411/411 [==============================] - 310s 754ms/step - loss: 4.6111 - accuracy: 0.0281 - val_loss: 4.6498 - val_accuracy: 0.0219
Epoch 4/20
411/411 [==============================] - 309s 750ms/step - loss: 4.4431 - accuracy: 0.0373 - val_loss: 4.3622 - val_accuracy: 0.0460
Epoch 5/20
411/411 [==============================] - 307s 747ms/step - loss: 4.3068 - accuracy: 0.0495 - val_loss: 4.2653 - val_accuracy: 0.0518
Epoch 6/20
411/411 [==============================] - 308s 748ms/step - loss: 4.1833 - accuracy: 0.0588 - val_loss: 4.2922 - val_accuracy: 0.0515
Epoch 7/20
411/411 [==============================] - 308s 749ms/step - loss: 4.0663 - accuracy: 0.0762 - val_loss: 4.1679 -

In [ ]:
#We can see major overfitting from epochs 11 on. Lets reduce filtering on this model and see about model improvement.

In [56]:
#model AlexNet_2 using less filters 
AlexNet_2 = models.Sequential()

# Layer 1: Convolutional layer with 64 filters of size 11x11, followed by max pooling
AlexNet_2.add(layers.Conv2D(64, kernel_size=(11,11),strides=4, activation='relu', padding='same', input_shape=(224, 224, 3)))
AlexNet_2.add(layers.MaxPooling2D(pool_size=(3, 3),strides=2))

# Layer 2: Convolutional layer with 192 filters of size 5x5, followed by max pooling
AlexNet_2.add(layers.Conv2D(192, kernel_size=(5,5),padding='same', activation='relu'))
AlexNet_2.add(layers.MaxPooling2D(pool_size=(3, 3),strides=2))

# Layer 3: Convolutional layers with reduced number of filters
AlexNet_2.add(layers.Conv2D(384, kernel_size=(3,3),padding='same', activation='relu'))
AlexNet_2.add(layers.Conv2D(256, kernel_size=(3,3),padding='same', activation='relu'))

AlexNet_2.add(layers.MaxPooling2D(pool_size=(3, 3),strides=2))

# Flatten the feature maps to feed into fully connected layers
AlexNet_2.add(layers.Flatten())

# Layer 4: Fully connected layer with reduced number of neurons
AlexNet_2.add(layers.Dense(2048, activation='relu'))
AlexNet_2.add(layers.Dropout(.5))

# Layer 5: Fully connected layer with reduced number of neurons
AlexNet_2.add(layers.Dense(1024, activation='relu'))
AlexNet_2.add(layers.Dropout(.5))

# Output layer: Fully connected layer with num_classes neurons (e.g., 3 )
AlexNet_2.add(layers.Dense(120, activation='softmax'))

# Compile model
AlexNet_2.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
AlexNet_2.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_68 (Conv2D)          (None, 56, 56, 64)        23296     
                                                                 
 max_pooling2d_59 (MaxPooli  (None, 27, 27, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_69 (Conv2D)          (None, 27, 27, 192)       307392    
                                                                 
 max_pooling2d_60 (MaxPooli  (None, 13, 13, 192)       0         
 ng2D)                                                           
                                                                 
 conv2d_70 (Conv2D)          (None, 13, 13, 384)       663936    
                                                                 
 conv2d_71 (Conv2D)          (None, 13, 13, 256)     

In [58]:
#fit the Model
history = AlexNet_2.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
359/359 [==============================] - 162s 451ms/step - loss: 4.7826 - accuracy: 0.0118 - val_loss: 4.7745 - val_accuracy: 0.0101
Epoch 2/20
359/359 [==============================] - 160s 445ms/step - loss: 4.7536 - accuracy: 0.0159 - val_loss: 4.7299 - val_accuracy: 0.0171
Epoch 3/20
359/359 [==============================] - 162s 451ms/step - loss: 4.6914 - accuracy: 0.0176 - val_loss: 4.6266 - val_accuracy: 0.0247
Epoch 4/20
359/359 [==============================] - 159s 443ms/step - loss: 4.6060 - accuracy: 0.0259 - val_loss: 4.5299 - val_accuracy: 0.0307
Epoch 5/20
359/359 [==============================] - 162s 450ms/step - loss: 4.4771 - accuracy: 0.0332 - val_loss: 4.4079 - val_accuracy: 0.0418
Epoch 6/20
359/359 [==============================] - 161s 447ms/step - loss: 4.3688 - accuracy: 0.0407 - val_loss: 4.3217 - val_accuracy: 0.0453
Epoch 7/20
359/359 [==============================] - 163s 453ms/step - loss: 4.2775 - accuracy: 0.0489 - val_loss: 4.1947 -

In [59]:
#Save The Model
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(AlexNet_2, "../Project4/AlexNet_2")
AlexNet_2.save("AlexNet_2.keras")

In [103]:
from keras.preprocessing import image
import numpy as np
AlexNet_2 = tf.keras.models.load_model('../Project4/AlexNet_2/AlexNet_2.keras')
# Load and preprocess the image
img_path = '../Project4/split_data/test/n02091032-Italian_greyhound/n02091032_1360.jpg'  
img = image.load_img(img_path, target_size=(224, 224))  # Rescale the image to match the input shape of the model
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize the pixel values


# Predict on the image (assuming img_array is the preprocessed image as shown in the previous example)
predictions = AlexNet_2.predict(img_array)

# Get the top N predicted class indices and probabilities
top_N = 5  # Change this to the number of top predictions you want to display
top_indices = np.argsort(predictions[0])[::-1][:top_N]
top_probabilities = predictions[0][top_indices]

# Decode the top predictions
top_labels = [classes[i] for i in top_indices]

# Print the top predictions along with their probabilities
for label, probability in zip(top_labels, top_probabilities):
    print(f'Dog Breed: {label}, Probability: {probability:.2%}')


1/1 [==============================] - 0s 424ms/step
Dog Breed: n02097658-silky_terrier, Probability: 0.93%
Dog Breed: n02086079-Pekinese, Probability: 0.91%
Dog Breed: n02090622-borzoi, Probability: 0.91%
Dog Breed: n02105505-komondor, Probability: 0.90%
Dog Breed: n02097474-Tibetan_terrier, Probability: 0.90%


In [31]:
from tensorflow.keras import callbacks

#Define the CNN simple_vgg
simple_vgg = models.Sequential()

#Convolutional layers
simple_vgg.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
simple_vgg.add(layers.BatchNormalization())
simple_vgg.add(layers.MaxPooling2D((2, 2)))
simple_vgg.add(layers.Conv2D(64, (3, 3), activation='relu'))
simple_vgg.add(layers.BatchNormalization())
simple_vgg.add(layers.MaxPooling2D((2, 2)))
simple_vgg.add(layers.Conv2D(128, (3, 3), activation='relu'))
simple_vgg.add(layers.BatchNormalization())
simple_vgg.add(layers.MaxPooling2D((2, 2)))
simple_vgg.add(layers.Conv2D(128, (3, 3), activation='relu'))
simple_vgg.add(layers.BatchNormalization())
simple_vgg.add(layers.MaxPooling2D((2, 2)))

# Flatten layer
simple_vgg.add(layers.Flatten())

# Dense layers
simple_vgg.add(layers.Dense(512, activation='relu'))
simple_vgg.add(layers.Dropout(0.5))  # Dropout for regularization
simple_vgg.add(layers.Dense(120, activation='softmax'))  # 120 output classes for 120 dog breeds

# Compile the simple_vgg
optimizer = optimizers.RMSprop(learning_rate=1e-4)
loss_function = 'sparse_categorical_crossentropy'

# Adding Early Stopping to prevent overfitting
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Adding Model Checkpoint to save the best model during training
model_checkpoint = callbacks.ModelCheckpoint("best_model.h5", save_best_only=True)

simple_vgg.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

# Print the simple_vgg summary
simple_vgg.summary()

#Since on every model ive seen major over fitting of data we implement some batch normalization and early stopping

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_45 (Conv2D)          (None, 126, 126, 32)      896       
                                                                 
 batch_normalization_16 (Ba  (None, 126, 126, 32)      128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_43 (MaxPooli  (None, 63, 63, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_46 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_17 (Ba  (None, 61, 61, 64)        256       
 tchNormalization)                                               
                                                     

In [32]:
#fit the simple_vgg from image generator
history = simple_vgg.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=30,
            validation_data=val_rescale_ds
)

Epoch 1/30
359/359 [==============================] - 108s 299ms/step - loss: 4.9890 - accuracy: 0.0166 - val_loss: 6.2098 - val_accuracy: 0.0153
Epoch 2/30
359/359 [==============================] - 107s 297ms/step - loss: 4.5946 - accuracy: 0.0358 - val_loss: 4.5485 - val_accuracy: 0.0442
Epoch 3/30
359/359 [==============================] - 107s 299ms/step - loss: 4.3229 - accuracy: 0.0672 - val_loss: 4.4424 - val_accuracy: 0.0550
Epoch 4/30
359/359 [==============================] - 105s 291ms/step - loss: 4.0350 - accuracy: 0.1063 - val_loss: 4.3882 - val_accuracy: 0.0651
Epoch 5/30
359/359 [==============================] - 105s 292ms/step - loss: 3.7035 - accuracy: 0.1542 - val_loss: 4.3528 - val_accuracy: 0.0752
Epoch 6/30
359/359 [==============================] - 106s 296ms/step - loss: 3.3537 - accuracy: 0.2138 - val_loss: 4.3175 - val_accuracy: 0.0791
Epoch 7/30
359/359 [==============================] - 105s 292ms/step - loss: 2.9622 - accuracy: 0.2840 - val_loss: 4.3368 -

In [34]:

tfjs.converters.save_keras_model(simple_vgg, "../Project4/")
simple_vgg.save("simple_vgg.keras")

/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [106]:
from keras.preprocessing import image
import numpy as np
simple_vgg = tf.keras.models.load_model('../Project4/simple_vgg/simple_vgg.keras')
# Load and preprocess the image
img_path = '../Project4/split_data/test/n02091032-Italian_greyhound/n02091032_1360.jpg'  
img = image.load_img(img_path, target_size=(128, 128))  # Rescale the image to match the input shape of the model
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize the pixel values


# Predict on the image (assuming img_array is the preprocessed image as shown in the previous example)
predictions = simple_vgg.predict(img_array)

# Get the top N predicted class indices and probabilities
top_N = 5  # Change this to the number of top predictions you want to display
top_indices = np.argsort(predictions[0])[::-1][:top_N]
top_probabilities = predictions[0][top_indices]

# Decode the top predictions
top_labels = [classes[i] for i in top_indices]

# Print the top predictions along with their probabilities
for label, probability in zip(top_labels, top_probabilities):
    print(f'Dog Breed: {label}, Probability: {probability:.2%}')


1/1 [==============================] - 1s 571ms/step
Dog Breed: n02109047-Great_Dane, Probability: 1.05%
Dog Breed: n02107142-Doberman, Probability: 0.99%
Dog Breed: n02112350-keeshond, Probability: 0.98%
Dog Breed: n02100583-vizsla, Probability: 0.96%
Dog Breed: n02110627-affenpinscher, Probability: 0.96%


In [ ]:
#We can see major overfitting from epochs 5 on.